# 🧠 Part 1: Building Optimized CNN from Scratch
## גרסה מיוטבת - מהירה + פחות overfitting!

---

### ✨ שיפורים בגרסה הזו:
1. **מודל קל יותר** - 3 conv layers אבל פחות channels (16→32→64)
2. **dropout חזק יותר** - 0.6 במקום 0.5 למניעת overfitting
3. **פחות epochs** - 25 במקום 50 (חיסכון זמן!)
4. **Early Stopping** - עצירה אוטומטית אחרי 5 epochs ללא שיפור
5. **3 FC layers** - מעבר הדרגתי יותר (256→128→4)

### 📊 תוצאות צפויות:
- ⏱️ זמן אימון: ~20-30 דקות (במקום 2+ שעות!)
- 📉 פחות overfitting
- 🎯 accuracy דומה או טוב יותר

## 1️⃣ ייבוא ספריות

In [ ]:
import os
import sys
from pathlib import Path
import json
import time

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
import torchvision
from torchvision import transforms, datasets
from torchvision.datasets import ImageFolder

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm

sys.path.append(str(Path.cwd().parent / 'src'))
import config

print("✅ All libraries imported!")
print(f"🖥️  Device: {config.DEVICE}")
print(f"📦 Batch size: {config.BATCH_SIZE}")
print(f"📈 Learning rate: {config.LEARNING_RATE}")
print(f"🔢 Max Epochs: {config.NUM_EPOCHS}")
print(f"⏸️  Early Stopping Patience: {config.PATIENCE}")

## 2️⃣ טעינת Dataset Splits

In [ ]:
splits_path = config.RESULTS_DIR / 'dataset_splits.json'

with open(splits_path, 'r') as f:
    splits_data = json.load(f)

train_indices = splits_data['train_indices']
val_indices = splits_data['val_indices']
test_indices = splits_data['test_indices']
class_names = splits_data['class_names']
num_classes = splits_data['num_classes']

print("✅ Splits loaded!")
print(f"📊 Train: {len(train_indices)}, Val: {len(val_indices)}, Test: {len(test_indices)}")

## 3️⃣ יצירת DataLoaders

In [ ]:
IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD = [0.229, 0.224, 0.225]

train_transforms = transforms.Compose([
    transforms.Resize((config.IMAGE_SIZE + 32, config.IMAGE_SIZE + 32)),
    transforms.RandomCrop((config.IMAGE_SIZE, config.IMAGE_SIZE)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=20),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
])

val_test_transforms = transforms.Compose([
    transforms.Resize((config.IMAGE_SIZE, config.IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
])

data_path = config.DATA_PATH

train_dataset_full = ImageFolder(root=str(data_path), transform=train_transforms)
train_dataset = Subset(train_dataset_full, train_indices)

val_dataset_full = ImageFolder(root=str(data_path), transform=val_test_transforms)
val_dataset = Subset(val_dataset_full, val_indices)

train_loader = DataLoader(train_dataset, batch_size=config.BATCH_SIZE, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=config.BATCH_SIZE, shuffle=False, num_workers=2)

print(f"✅ DataLoaders ready! Train: {len(train_loader)} batches, Val: {len(val_loader)} batches")

## 4️⃣ SimpleCNN - גרסה מיוטבת! 🏗️

### 🔥 השינויים:
- **Channels:** 16→32→64 (במקום 32→64)
- **Conv layers:** 3 במקום 2 (יותר עומק, פחות רוחב)
- **FC layers:** 256→128→4 (במקום 512→4)
- **Dropout:** 0.6 ו-0.5 (במקום 0.5 בלבד)

**תוצאה:** ~12M parameters במקום 102M! ⚡

In [ ]:
class SimpleCNN(nn.Module):
    def __init__(self, num_classes=4):
        super(SimpleCNN, self).__init__()
        
        # Block 1: 3→16
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(2, 2)  # 224→112
        
        # Block 2: 16→32
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(2, 2)  # 112→56
        
        # Block 3: 32→64
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.relu3 = nn.ReLU()
        self.pool3 = nn.MaxPool2d(2, 2)  # 56→28
        
        # Flatten: 64×28×28 = 50,176
        self.flatten = nn.Flatten()
        
        # FC1: 50176→256
        self.fc1 = nn.Linear(64 * 28 * 28, 256)
        self.relu4 = nn.ReLU()
        self.dropout1 = nn.Dropout(p=0.6)  # Dropout חזק
        
        # FC2: 256→128
        self.fc2 = nn.Linear(256, 128)
        self.relu5 = nn.ReLU()
        self.dropout2 = nn.Dropout(p=0.5)
        
        # FC3: 128→4
        self.fc3 = nn.Linear(128, num_classes)
        
    def forward(self, x):
        # Conv blocks
        x = self.pool1(self.relu1(self.conv1(x)))  # [B,3,224,224]→[B,16,112,112]
        x = self.pool2(self.relu2(self.conv2(x)))  # [B,16,112,112]→[B,32,56,56]
        x = self.pool3(self.relu3(self.conv3(x)))  # [B,32,56,56]→[B,64,28,28]
        
        # FC layers
        x = self.flatten(x)                        # [B,64,28,28]→[B,50176]
        x = self.dropout1(self.relu4(self.fc1(x)))  # [B,50176]→[B,256]
        x = self.dropout2(self.relu5(self.fc2(x)))  # [B,256]→[B,128]
        x = self.fc3(x)                            # [B,128]→[B,4]
        
        return x

model = SimpleCNN(num_classes=num_classes).to(config.DEVICE)

total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("✅ SimpleCNN created!")
print(f"🔢 Total parameters: {total_params:,}")
print(f"💾 Model size: ~{total_params * 4 / (1024**2):.1f} MB")
print(f"\n{model}")

## 5️⃣ Loss & Optimizer

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=config.LEARNING_RATE, weight_decay=config.WEIGHT_DECAY)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

print("✅ Optimizer ready!")

## 6️⃣ פונקציות Train & Validate

In [ ]:
def train_one_epoch(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    pbar = tqdm(dataloader, desc='Training', leave=False)
    for images, labels in pbar:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        running_loss += loss.item()
        
        pbar.set_postfix({'loss': f'{loss.item():.4f}', 'acc': f'{100*correct/total:.2f}%'})
    
    return running_loss / len(dataloader), 100 * correct / total

def validate(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        pbar = tqdm(dataloader, desc='Validation', leave=False)
        for images, labels in pbar:
            images, labels = images.to(device), labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            running_loss += loss.item()
            
            pbar.set_postfix({'loss': f'{loss.item():.4f}', 'acc': f'{100*correct/total:.2f}%'})
    
    return running_loss / len(dataloader), 100 * correct / total

print("✅ Training functions ready!")

## 7️⃣ לולאת האימון + Early Stopping 🚀

In [ ]:
history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}
best_val_acc = 0.0
best_model_path = config.MODELS_DIR / 'simple_cnn_optimized_best.pth'

# Early Stopping
patience = config.PATIENCE
patience_counter = 0

print("🚀 Starting training...")
print(f"   Max Epochs: {config.NUM_EPOCHS}")
print(f"   Early Stopping: {patience} epochs patience")
print(f"   Device: {config.DEVICE}")
print("\n" + "="*80)

start_time = time.time()

for epoch in range(config.NUM_EPOCHS):
    epoch_start = time.time()
    
    # Train
    train_loss, train_acc = train_one_epoch(model, train_loader, criterion, optimizer, config.DEVICE)
    
    # Validate
    val_loss, val_acc = validate(model, val_loader, criterion, config.DEVICE)
    
    # Update scheduler
    scheduler.step()
    
    # Save history
    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)
    history['val_loss'].append(val_loss)
    history['val_acc'].append(val_acc)
    
    epoch_time = time.time() - epoch_start
    
    # Print
    print(f"Epoch [{epoch+1:2d}/{config.NUM_EPOCHS}] | "
          f"Train: {train_loss:.4f} / {train_acc:5.2f}% | "
          f"Val: {val_loss:.4f} / {val_acc:5.2f}% | "
          f"{epoch_time:.1f}s", end='')
    
    # Save best model & Early Stopping
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        patience_counter = 0
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_acc': val_acc,
            'val_loss': val_loss,
        }, best_model_path)
        print(f" 💾 BEST!")
    else:
        patience_counter += 1
        print(f" ⏳ ({patience_counter}/{patience})")
        
        if patience_counter >= patience:
            print(f"\n\n⚠️  Early stopping after {epoch+1} epochs (no improvement for {patience} epochs)")
            break

total_time = time.time() - start_time
print("\n" + "="*80)
print(f"✅ Training completed!")
print(f"   Time: {total_time/60:.1f} minutes")
print(f"   Best Val Acc: {best_val_acc:.2f}%")
print(f"   Saved to: {best_model_path}")

## 8️⃣ Training Curves

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 5))
epochs_range = range(1, len(history['train_loss']) + 1)

# Loss
axes[0].plot(epochs_range, history['train_loss'], 'b-o', label='Train', linewidth=2)
axes[0].plot(epochs_range, history['val_loss'], 'r-o', label='Val', linewidth=2)
axes[0].set_xlabel('Epoch', fontsize=12)
axes[0].set_ylabel('Loss', fontsize=12)
axes[0].set_title('Loss', fontsize=14, fontweight='bold')
axes[0].legend(fontsize=11)
axes[0].grid(True, alpha=0.3)

# Accuracy
axes[1].plot(epochs_range, history['train_acc'], 'b-o', label='Train', linewidth=2)
axes[1].plot(epochs_range, history['val_acc'], 'r-o', label='Val', linewidth=2)
axes[1].set_xlabel('Epoch', fontsize=12)
axes[1].set_ylabel('Accuracy (%)', fontsize=12)
axes[1].set_title('Accuracy', fontsize=14, fontweight='bold')
axes[1].legend(fontsize=11)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(config.PLOTS_DIR / '06_optimized_cnn_training_curves.png', dpi=300, bbox_inches='tight')
plt.show()

print("💾 Plot saved!")

## 9️⃣ ניתוח תוצאות

In [ ]:
final_train_acc = history['train_acc'][-1]
final_val_acc = history['val_acc'][-1]
acc_gap = final_train_acc - final_val_acc

print("="*60)
print("📊 Final Results")
print("="*60)
print(f"🎯 Best Val Accuracy: {best_val_acc:.2f}%")
print(f"📈 Final Train Acc: {final_train_acc:.2f}%")
print(f"📉 Final Val Acc: {final_val_acc:.2f}%")
print(f"⚠️  Overfitting Gap: {acc_gap:.2f}%")
print(f"⏱️  Training Time: {total_time/60:.1f} minutes")
print(f"🔢 Parameters: {total_params:,}")

if acc_gap < 5:
    print("\n✅ Excellent! Minimal overfitting.")
elif acc_gap < 10:
    print("\n👍 Good! Some overfitting but acceptable.")
else:
    print("\n⚠️  Still overfitting. Consider more regularization.")

print("="*60)

## 🔟 שמירת תוצאות

In [ ]:
results = {
    'model_name': 'SimpleCNN_Optimized',
    'num_epochs_ran': len(history['train_loss']),
    'max_epochs': config.NUM_EPOCHS,
    'batch_size': config.BATCH_SIZE,
    'learning_rate': config.LEARNING_RATE,
    'early_stopping_patience': patience,
    'best_val_acc': float(best_val_acc),
    'final_train_acc': float(final_train_acc),
    'final_val_acc': float(final_val_acc),
    'overfitting_gap': float(acc_gap),
    'total_params': total_params,
    'training_time_minutes': total_time / 60,
    'history': history
}

results_path = config.LOGS_DIR / 'simple_cnn_optimized_results.json'
with open(results_path, 'w') as f:
    json.dump(results, f, indent=2)

print(f"💾 Results saved to: {results_path}")

## 📝 סיכום

### ✨ מה שיפרנו:
1. **מהירות:** פחות פרמטרים = אימון מהר יותר
2. **Overfitting:** Dropout חזק + Early stopping
3. **יעילות:** עצירה אוטומטית כשלא משתפר

### 🚀 הצעד הבא:
במחברת הבאה נוסיף **Batch Normalization** לשיפור נוסף!